In [1]:
import requests
import numpy as np
import pandas as pd

company = 'GOOG'
demo = 'b549de94717a98e521c8cbb270f00ad5'
Incm_St = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={demo}').json()

#We will get revenue growth, that we will use to estimate future sales
revenue_g = 0.02
# Get Net Income
Net_Income = Incm_St[0]['netIncome']

#Converting the Net_Income to Pandas Data Frame for better visualization 
income_statement = pd.DataFrame.from_dict(Incm_St[0], orient = 'index')
#print(income_statement)
#Cleaning out excess datat that gets imported from the API
income_statement = income_statement[5:29]
income_statement.columns = ['Current_Year']
income_statement['As_%_revenue'] = income_statement/income_statement.iloc[0]

#Forecasting the next 5 years of Income Statement based on revenue growth 
income_statement['Next_Year_1'] =  (income_statement['Current_Year']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 
income_statement['Next_Year_2'] =  (income_statement['Next_Year_1']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 
income_statement['Next_Year_3'] =  (income_statement['Next_Year_2']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 
income_statement['Next_Year_4'] =  (income_statement['Next_Year_3']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 
income_statement['Next_Year_5'] =  (income_statement['Next_Year_4']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 

# Get Balance Sheet 
# Following the same procedure as above when we imported Income Statement 
Blnc_Sht = BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={demo}').json()
Balance_Sheet = pd.DataFrame.from_dict(Blnc_Sht[0], orient='index')
Balance_Sheet.columns = ['Current_Year']
Balance_Sheet = Balance_Sheet[5:-2]
Balance_Sheet['As_%_revnues'] = Balance_Sheet/ income_statement['Current_Year'].iloc[0]

#Forcasting for the next 5 years now
Balance_Sheet['Next_Year_1'] =  income_statement['Next_Year_1']['revenue'] * Balance_Sheet['As_%_revnues']
Balance_Sheet['Next_Year_2'] =  income_statement['Next_Year_2']['revenue'] * Balance_Sheet['As_%_revnues']
Balance_Sheet['Next_Year_3'] =  income_statement['Next_Year_3']['revenue'] * Balance_Sheet['As_%_revnues'] 
Balance_Sheet['Next_Year_4'] =  income_statement['Next_Year_4']['revenue'] * Balance_Sheet['As_%_revnues'] 
Balance_Sheet['Next_Year_5'] =  income_statement['Next_Year_5']['revenue'] * Balance_Sheet['As_%_revnues']

# Getting Cash Flows 
FCF_Forecast = {}
FCF_Forecast['Next_Year_1'] = {}
FCF_Forecast['Next_Year_1']['netIncome'] = income_statement['Next_Year_1']['netIncome']
FCF_Forecast['Next_Year_1']['Depreciation '] = income_statement['Next_Year_1']['depreciationAndAmortization'] - income_statement['Current_Year']['depreciationAndAmortization']
FCF_Forecast['Next_Year_1']['Receivables '] = Balance_Sheet['Next_Year_1']['netReceivables'] - Balance_Sheet['Current_Year']['netReceivables']
FCF_Forecast['Next_Year_1']['Inventory'] = Balance_Sheet['Next_Year_1']['inventory'] - Balance_Sheet['Current_Year']['inventory']
FCF_Forecast['Next_Year_1']['Payables '] = Balance_Sheet['Next_Year_1']['accountPayables'] - Balance_Sheet['Current_Year']['accountPayables']
FCF_Forecast['Next_Year_1']['CF_operations'] = FCF_Forecast['Next_Year_1']['netIncome'] + FCF_Forecast['Next_Year_1']['Depreciation '] + (FCF_Forecast['Next_Year_1']['Receivables '] * -1) + (FCF_Forecast['Next_Year_1']['Inventory'] *-1) + FCF_Forecast['Next_Year_1']['Payables ']
FCF_Forecast['Next_Year_1']['CAPEX'] = Balance_Sheet['Next_Year_1']['propertyPlantEquipmentNet'] - Balance_Sheet['Current_Year']['propertyPlantEquipmentNet'] + income_statement['Next_Year_1']['depreciationAndAmortization']

FCF_Forecast['Next_Year_1']['FCF'] = FCF_Forecast['Next_Year_1']['CAPEX'] + FCF_Forecast['Next_Year_1']['CF_operations']


FCF_Forecast['Next_Year_2'] = {}
FCF_Forecast['Next_Year_2']['netIncome'] = income_statement['Next_Year_2']['netIncome']

FCF_Forecast['Next_Year_2']['Depreciation '] = income_statement['Next_Year_2']['depreciationAndAmortization'] - income_statement['Next_Year_1']['depreciationAndAmortization']
FCF_Forecast['Next_Year_2']['Receivables '] = Balance_Sheet['Next_Year_2']['netReceivables'] - Balance_Sheet['Next_Year_1']['netReceivables']
FCF_Forecast['Next_Year_2']['Inventory'] = Balance_Sheet['Next_Year_2']['inventory'] - Balance_Sheet['Next_Year_1']['inventory']
FCF_Forecast['Next_Year_2']['Payables '] = Balance_Sheet['Next_Year_2']['accountPayables'] - Balance_Sheet['Next_Year_1']['accountPayables']
FCF_Forecast['Next_Year_2']['CF_operations'] = FCF_Forecast['Next_Year_2']['netIncome'] + FCF_Forecast['Next_Year_2']['Depreciation '] + (FCF_Forecast['Next_Year_2']['Receivables '] * -1) + (FCF_Forecast['Next_Year_2']['Inventory'] *-1) + FCF_Forecast['Next_Year_2']['Payables ']
FCF_Forecast['Next_Year_2']['CAPEX'] = Balance_Sheet['Next_Year_2']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_1']['propertyPlantEquipmentNet'] + income_statement['Next_Year_2']['depreciationAndAmortization']
FCF_Forecast['Next_Year_2']['FCF'] = FCF_Forecast['Next_Year_2']['CAPEX'] + FCF_Forecast['Next_Year_2']['CF_operations']


FCF_Forecast['Next_Year_3'] = {}
FCF_Forecast['Next_Year_3']['netIncome'] = income_statement['Next_Year_3']['netIncome']

FCF_Forecast['Next_Year_3']['Depreciation '] = income_statement['Next_Year_3']['depreciationAndAmortization'] - income_statement['Next_Year_2']['depreciationAndAmortization']
FCF_Forecast['Next_Year_3']['Receivables '] = Balance_Sheet['Next_Year_3']['netReceivables'] - Balance_Sheet['Next_Year_2']['netReceivables']
FCF_Forecast['Next_Year_3']['Inventory'] = Balance_Sheet['Next_Year_3']['inventory'] - Balance_Sheet['Next_Year_2']['inventory']
FCF_Forecast['Next_Year_3']['Payables '] = Balance_Sheet['Next_Year_3']['accountPayables'] - Balance_Sheet['Next_Year_2']['accountPayables']
FCF_Forecast['Next_Year_3']['CF_operations'] = FCF_Forecast['Next_Year_3']['netIncome'] + FCF_Forecast['Next_Year_3']['Depreciation '] + (FCF_Forecast['Next_Year_3']['Receivables '] * -1) + (FCF_Forecast['Next_Year_3']['Inventory'] *-1) + FCF_Forecast['Next_Year_3']['Payables ']
FCF_Forecast['Next_Year_3']['CAPEX'] = Balance_Sheet['Next_Year_3']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_2']['propertyPlantEquipmentNet'] + income_statement['Next_Year_3']['depreciationAndAmortization']
FCF_Forecast['Next_Year_3']['FCF'] = FCF_Forecast['Next_Year_3']['CAPEX'] + FCF_Forecast['Next_Year_3']['CF_operations']


FCF_Forecast['Next_Year_4'] = {}
FCF_Forecast['Next_Year_4']['netIncome'] = income_statement['Next_Year_4']['netIncome']

FCF_Forecast['Next_Year_4']['Depreciation '] = income_statement['Next_Year_4']['depreciationAndAmortization'] - income_statement['Next_Year_3']['depreciationAndAmortization']
FCF_Forecast['Next_Year_4']['Receivables '] = Balance_Sheet['Next_Year_4']['netReceivables'] - Balance_Sheet['Next_Year_3']['netReceivables']
FCF_Forecast['Next_Year_4']['Inventory'] = Balance_Sheet['Next_Year_4']['inventory'] - Balance_Sheet['Next_Year_3']['inventory']
FCF_Forecast['Next_Year_4']['Payables '] = Balance_Sheet['Next_Year_4']['accountPayables'] - Balance_Sheet['Next_Year_3']['accountPayables']
FCF_Forecast['Next_Year_4']['CF_operations'] = FCF_Forecast['Next_Year_4']['netIncome'] + FCF_Forecast['Next_Year_4']['Depreciation '] + (FCF_Forecast['Next_Year_4']['Receivables '] * -1) + (FCF_Forecast['Next_Year_4']['Inventory'] *-1) + FCF_Forecast['Next_Year_4']['Payables ']
FCF_Forecast['Next_Year_4']['CAPEX'] = Balance_Sheet['Next_Year_4']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_3']['propertyPlantEquipmentNet'] + income_statement['Next_Year_4']['depreciationAndAmortization']
FCF_Forecast['Next_Year_4']['FCF'] = FCF_Forecast['Next_Year_4']['CAPEX'] + FCF_Forecast['Next_Year_4']['CF_operations']

FCF_Forecast['Next_Year_5'] = {}
FCF_Forecast['Next_Year_5']['netIncome'] = income_statement['Next_Year_5']['netIncome']

FCF_Forecast['Next_Year_5']['Depreciation '] = income_statement['Next_Year_5']['depreciationAndAmortization'] - income_statement['Next_Year_4']['depreciationAndAmortization']
FCF_Forecast['Next_Year_5']['Receivables '] = Balance_Sheet['Next_Year_5']['netReceivables'] - Balance_Sheet['Next_Year_4']['netReceivables']
FCF_Forecast['Next_Year_5']['Inventory'] = Balance_Sheet['Next_Year_5']['inventory'] - Balance_Sheet['Next_Year_4']['inventory']
FCF_Forecast['Next_Year_5']['Payables '] = Balance_Sheet['Next_Year_5']['accountPayables'] - Balance_Sheet['Next_Year_4']['accountPayables']
FCF_Forecast['Next_Year_5']['CF_operations'] = FCF_Forecast['Next_Year_5']['netIncome'] + FCF_Forecast['Next_Year_5']['Depreciation '] + (FCF_Forecast['Next_Year_5']['Receivables '] * -1) + (FCF_Forecast['Next_Year_5']['Inventory'] *-1) + FCF_Forecast['Next_Year_5']['Payables ']
FCF_Forecast['Next_Year_5']['CAPEX'] = Balance_Sheet['Next_Year_5']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_4']['propertyPlantEquipmentNet'] + income_statement['Next_Year_5']['depreciationAndAmortization']
FCF_Forecast['Next_Year_5']['FCF'] = FCF_Forecast['Next_Year_5']['CAPEX'] + FCF_Forecast['Next_Year_5']['CF_operations']

#converting the dictionary to Panda Dataset for better visualization
FCF_Forecast_data = pd.DataFrame.from_dict(FCF_Forecast, orient ='columns')
#Seperate the numbers to thousands for better formatting
pd.options.display.float_format = '{:,.0f}'.format

# NPV OF CASH FLOWS
# The values below (WACC and PG will be simulated using Monte Carlo Simulation)
perpetual_growth = 0.02 #assuming 
WACC = 0.09 # assuming

fcf_lst = FCF_Forecast_data.iloc[-1].values.tolist()
npv = np.npv(WACC, fcf_lst)

# Getting total number of a Company's Shares
number_of_shares = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{company}?apikey={demo}').json()
number_of_shares = number_of_shares[0]['numberOfShares']

#Calculating the Terminal value 
terminal_value = (FCF_Forecast['Next_Year_5']['FCF']*(1+perpetual_growth))/(WACC-perpetual_growth)
terminal_value
# Calculating the share price of the company 
terminal_value_discounted = terminal_value/(1-WACC)**4
company_value = terminal_value_discounted + npv
share_value = company_value/number_of_shares
print("SHARE VALUE OF", company, '$',share_value)


SHARE VALUE OF GOOG $ 1817.9311107882338


In [2]:
# Get Balance Sheet 
# Following the same procedure as above when we imported Income Statement 
Blnc_Sht = BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={demo}').json()
Balance_Sheet = pd.DataFrame.from_dict(Blnc_Sht[0], orient='index')
Balance_Sheet.columns = ['Current_Year']
Balance_Sheet = Balance_Sheet[5:-2]
Balance_Sheet['As_%_revnues'] = Balance_Sheet/ income_statement['Current_Year'].iloc[0]

#Forcasting for the next 5 years now
Balance_Sheet['Next_Year_1'] =  income_statement['Next_Year_1']['revenue'] * Balance_Sheet['As_%_revnues']
Balance_Sheet['Next_Year_2'] =  income_statement['Next_Year_2']['revenue'] * Balance_Sheet['As_%_revnues']
Balance_Sheet['Next_Year_3'] =  income_statement['Next_Year_3']['revenue'] * Balance_Sheet['As_%_revnues'] 
Balance_Sheet['Next_Year_4'] =  income_statement['Next_Year_4']['revenue'] * Balance_Sheet['As_%_revnues'] 
Balance_Sheet['Next_Year_5'] =  income_statement['Next_Year_5']['revenue'] * Balance_Sheet['As_%_revnues']

Balance_Sheet

,Current_Year,As_%_revnues,Next_Year_1,Next_Year_2,Next_Year_3,Next_Year_4,Next_Year_5
cashAndCashEquivalents,18498000000,0,"18,867,960,000","19,245,319,200","19,630,225,584","20,022,830,096","20,423,286,698"
shortTermInvestments,101177000000,1,"103,200,540,000","105,264,550,800","107,369,841,816","109,517,238,652","111,707,583,425"
cashAndShortTermInvestments,119675000000,1,"122,068,500,000","124,509,870,000","127,000,067,400","129,540,068,748","132,130,870,123"
netReceivables,27492000000,0,"28,041,840,000","28,602,676,800","29,174,730,336","29,758,224,943","30,353,389,442"
inventory,999000000,0,"1,018,980,000","1,039,359,600","1,060,146,792","1,081,349,728","1,102,976,722"
otherCurrentAssets,4412000000,0,"4,500,240,000","4,590,244,800","4,682,049,696","4,775,690,690","4,871,204,504"
totalCurrentAssets,152578000000,1,"155,629,560,000","158,742,151,200","161,916,994,224","165,155,334,108","168,458,440,791"
propertyPlantEquipmentNet,84587000000,1,"86,278,740,000","88,004,314,800","89,764,401,096","91,559,689,118","93,390,882,900"
goodwill,20624000000,0,"21,036,480,000","21,457,209,600","21,886,353,792","22,324,080,868","22,770,562,485"
intangibleAssets,1979000000,0,"2,018,580,000","2,058,951,600","2,100,130,632","2,142,133,245","2,184,975,910"


In [3]:
FCF_Forecast = {}
FCF_Forecast['Next_Year_1'] = {}
FCF_Forecast['Next_Year_1']['netIncome'] = income_statement['Next_Year_1']['netIncome']
FCF_Forecast['Next_Year_1']['Depreciation '] = income_statement['Next_Year_1']['depreciationAndAmortization'] - income_statement['Current_Year']['depreciationAndAmortization']
FCF_Forecast['Next_Year_1']['Receivables '] = Balance_Sheet['Next_Year_1']['netReceivables'] - Balance_Sheet['Current_Year']['netReceivables']
FCF_Forecast['Next_Year_1']['Inventory'] = Balance_Sheet['Next_Year_1']['inventory'] - Balance_Sheet['Current_Year']['inventory']
FCF_Forecast['Next_Year_1']['Payables '] = Balance_Sheet['Next_Year_1']['accountPayables'] - Balance_Sheet['Current_Year']['accountPayables']
FCF_Forecast['Next_Year_1']['CF_operations'] = FCF_Forecast['Next_Year_1']['netIncome'] + FCF_Forecast['Next_Year_1']['Depreciation '] + (FCF_Forecast['Next_Year_1']['Receivables '] * -1) + (FCF_Forecast['Next_Year_1']['Inventory'] *-1) + FCF_Forecast['Next_Year_1']['Payables ']
FCF_Forecast['Next_Year_1']['CAPEX'] = Balance_Sheet['Next_Year_1']['propertyPlantEquipmentNet'] - Balance_Sheet['Current_Year']['propertyPlantEquipmentNet'] + income_statement['Next_Year_1']['depreciationAndAmortization']

FCF_Forecast['Next_Year_1']['FCF'] = FCF_Forecast['Next_Year_1']['CAPEX'] + FCF_Forecast['Next_Year_1']['CF_operations']


FCF_Forecast['Next_Year_2'] = {}
FCF_Forecast['Next_Year_2']['netIncome'] = income_statement['Next_Year_2']['netIncome']

FCF_Forecast['Next_Year_2']['Depreciation '] = income_statement['Next_Year_2']['depreciationAndAmortization'] - income_statement['Next_Year_1']['depreciationAndAmortization']
FCF_Forecast['Next_Year_2']['Receivables '] = Balance_Sheet['Next_Year_2']['netReceivables'] - Balance_Sheet['Next_Year_1']['netReceivables']
FCF_Forecast['Next_Year_2']['Inventory'] = Balance_Sheet['Next_Year_2']['inventory'] - Balance_Sheet['Next_Year_1']['inventory']
FCF_Forecast['Next_Year_2']['Payables '] = Balance_Sheet['Next_Year_2']['accountPayables'] - Balance_Sheet['Next_Year_1']['accountPayables']
FCF_Forecast['Next_Year_2']['CF_operations'] = FCF_Forecast['Next_Year_2']['netIncome'] + FCF_Forecast['Next_Year_2']['Depreciation '] + (FCF_Forecast['Next_Year_2']['Receivables '] * -1) + (FCF_Forecast['Next_Year_2']['Inventory'] *-1) + FCF_Forecast['Next_Year_2']['Payables ']
FCF_Forecast['Next_Year_2']['CAPEX'] = Balance_Sheet['Next_Year_2']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_1']['propertyPlantEquipmentNet'] + income_statement['Next_Year_2']['depreciationAndAmortization']
FCF_Forecast['Next_Year_2']['FCF'] = FCF_Forecast['Next_Year_2']['CAPEX'] + FCF_Forecast['Next_Year_2']['CF_operations']


FCF_Forecast['Next_Year_3'] = {}
FCF_Forecast['Next_Year_3']['netIncome'] = income_statement['Next_Year_3']['netIncome']

FCF_Forecast['Next_Year_3']['Depreciation '] = income_statement['Next_Year_3']['depreciationAndAmortization'] - income_statement['Next_Year_2']['depreciationAndAmortization']
FCF_Forecast['Next_Year_3']['Receivables '] = Balance_Sheet['Next_Year_3']['netReceivables'] - Balance_Sheet['Next_Year_2']['netReceivables']
FCF_Forecast['Next_Year_3']['Inventory'] = Balance_Sheet['Next_Year_3']['inventory'] - Balance_Sheet['Next_Year_2']['inventory']
FCF_Forecast['Next_Year_3']['Payables '] = Balance_Sheet['Next_Year_3']['accountPayables'] - Balance_Sheet['Next_Year_2']['accountPayables']
FCF_Forecast['Next_Year_3']['CF_operations'] = FCF_Forecast['Next_Year_3']['netIncome'] + FCF_Forecast['Next_Year_3']['Depreciation '] + (FCF_Forecast['Next_Year_3']['Receivables '] * -1) + (FCF_Forecast['Next_Year_3']['Inventory'] *-1) + FCF_Forecast['Next_Year_3']['Payables ']
FCF_Forecast['Next_Year_3']['CAPEX'] = Balance_Sheet['Next_Year_3']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_2']['propertyPlantEquipmentNet'] + income_statement['Next_Year_3']['depreciationAndAmortization']
FCF_Forecast['Next_Year_3']['FCF'] = FCF_Forecast['Next_Year_3']['CAPEX'] + FCF_Forecast['Next_Year_3']['CF_operations']


FCF_Forecast['Next_Year_4'] = {}
FCF_Forecast['Next_Year_4']['netIncome'] = income_statement['Next_Year_4']['netIncome']

FCF_Forecast['Next_Year_4']['Depreciation '] = income_statement['Next_Year_4']['depreciationAndAmortization'] - income_statement['Next_Year_3']['depreciationAndAmortization']
FCF_Forecast['Next_Year_4']['Receivables '] = Balance_Sheet['Next_Year_4']['netReceivables'] - Balance_Sheet['Next_Year_3']['netReceivables']
FCF_Forecast['Next_Year_4']['Inventory'] = Balance_Sheet['Next_Year_4']['inventory'] - Balance_Sheet['Next_Year_3']['inventory']
FCF_Forecast['Next_Year_4']['Payables '] = Balance_Sheet['Next_Year_4']['accountPayables'] - Balance_Sheet['Next_Year_3']['accountPayables']
FCF_Forecast['Next_Year_4']['CF_operations'] = FCF_Forecast['Next_Year_4']['netIncome'] + FCF_Forecast['Next_Year_4']['Depreciation '] + (FCF_Forecast['Next_Year_4']['Receivables '] * -1) + (FCF_Forecast['Next_Year_4']['Inventory'] *-1) + FCF_Forecast['Next_Year_4']['Payables ']
FCF_Forecast['Next_Year_4']['CAPEX'] = Balance_Sheet['Next_Year_4']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_3']['propertyPlantEquipmentNet'] + income_statement['Next_Year_4']['depreciationAndAmortization']
FCF_Forecast['Next_Year_4']['FCF'] = FCF_Forecast['Next_Year_4']['CAPEX'] + FCF_Forecast['Next_Year_4']['CF_operations']

FCF_Forecast['Next_Year_5'] = {}
FCF_Forecast['Next_Year_5']['netIncome'] = income_statement['Next_Year_5']['netIncome']

FCF_Forecast['Next_Year_5']['Depreciation '] = income_statement['Next_Year_5']['depreciationAndAmortization'] - income_statement['Next_Year_4']['depreciationAndAmortization']
FCF_Forecast['Next_Year_5']['Receivables '] = Balance_Sheet['Next_Year_5']['netReceivables'] - Balance_Sheet['Next_Year_4']['netReceivables']
FCF_Forecast['Next_Year_5']['Inventory'] = Balance_Sheet['Next_Year_5']['inventory'] - Balance_Sheet['Next_Year_4']['inventory']
FCF_Forecast['Next_Year_5']['Payables '] = Balance_Sheet['Next_Year_5']['accountPayables'] - Balance_Sheet['Next_Year_4']['accountPayables']
FCF_Forecast['Next_Year_5']['CF_operations'] = FCF_Forecast['Next_Year_5']['netIncome'] + FCF_Forecast['Next_Year_5']['Depreciation '] + (FCF_Forecast['Next_Year_5']['Receivables '] * -1) + (FCF_Forecast['Next_Year_5']['Inventory'] *-1) + FCF_Forecast['Next_Year_5']['Payables ']
FCF_Forecast['Next_Year_5']['CAPEX'] = Balance_Sheet['Next_Year_5']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_4']['propertyPlantEquipmentNet'] + income_statement['Next_Year_5']['depreciationAndAmortization']
FCF_Forecast['Next_Year_5']['FCF'] = FCF_Forecast['Next_Year_5']['CAPEX'] + FCF_Forecast['Next_Year_5']['CF_operations']






In [4]:
#converting the dictionary to Panda Dataset for better visualization
FCF_Forecast_data = pd.DataFrame.from_dict(FCF_Forecast, orient ='columns')
#Seperate the numbers to thousands for better formatting
pd.options.display.float_format = '{:,.0f}'.format
FCF_Forecast_data

,Next_Year_1,Next_Year_2,Next_Year_3,Next_Year_4,Next_Year_5
CAPEX,"13,708,360,000","13,982,527,200","14,262,177,744","14,547,421,299","14,838,369,725"
CF_operations,"34,806,880,000","35,503,017,600","36,213,077,952","36,937,339,511","37,676,086,301"
Depreciation,"235,620,000","240,332,400","245,139,048","250,041,829","255,042,666"
FCF,"48,515,240,000","49,485,544,800","50,475,255,696","51,484,760,810","52,514,456,026"
Inventory,"19,980,000","20,379,600","20,787,192","21,202,936","21,626,995"
Payables,"111,220,000","113,444,400","115,713,288","118,027,554","120,388,105"
Receivables,"549,840,000","560,836,800","572,053,536","583,494,607","595,164,499"
netIncome,"35,029,860,000","35,730,457,200","36,445,066,344","37,173,967,671","37,917,447,024"


In [5]:
# NPV OF CASH FLOWS
# The values below (WACC and PG will be simulated using Monte Carlo Simulation)
perpetual_growth = 0.02 #assuming 
WACC = 0.09 # assuming

fcf_lst = FCF_Forecast_data.iloc[-1].values.tolist()
npv = np.npv(WACC, fcf_lst)
npv

154051978293.33905

In [6]:
# Getting total number of a Company's Shares
number_of_shares = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{company}?apikey={demo}').json()
number_of_shares = number_of_shares[0]['numberOfShares']

#Calculating the Terminal value 
terminal_value = (FCF_Forecast['Next_Year_5']['FCF']*(1+perpetual_growth))/(WACC-perpetual_growth)
terminal_value

terminal_value_discounted = terminal_value/(1-WACC)**4
company_value = terminal_value_discounted + npv
share_value = company_value/number_of_shares
print("SHARE VALUE OF", company, '$',share_value)

SHARE VALUE OF GOOG $ 1817.9311107882338


In [23]:
import requests
import numpy as np
import pandas as pd
demo = 'b549de94717a98e521c8cbb270f00ad5'

def Share_Price_FCF_Forc(company, revenue_g, perpetual_growth, WACC):
    #company = 'GOOG'
    Incm_St = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={demo}').json()

    #We will get revenue growth, that we will use to estimate future sales
    #revenue_g = 0.02
    # Get Net Income
    Net_Income = Incm_St[0]['netIncome']

    #Converting the Net_Income to Pandas Data Frame for better visualization 
    income_statement = pd.DataFrame.from_dict(Incm_St[0], orient = 'index')
    #print(income_statement)
    #Cleaning out excess datat that gets imported from the API
    income_statement = income_statement[5:29]
    income_statement.columns = ['Current_Year']
    income_statement['As_%_revenue'] = income_statement/income_statement.iloc[0]

    #Forecasting the next 5 years of Income Statement based on revenue growth 
    income_statement['Next_Year_1'] =  (income_statement['Current_Year']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 
    income_statement['Next_Year_2'] =  (income_statement['Next_Year_1']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 
    income_statement['Next_Year_3'] =  (income_statement['Next_Year_2']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 
    income_statement['Next_Year_4'] =  (income_statement['Next_Year_3']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 
    income_statement['Next_Year_5'] =  (income_statement['Next_Year_4']['revenue'] * (1+revenue_g)) * income_statement['As_%_revenue'] 

    # Get Balance Sheet 
    # Following the same procedure as above when we imported Income Statement 
    Blnc_Sht = BS = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={demo}').json()
    Balance_Sheet = pd.DataFrame.from_dict(Blnc_Sht[0], orient='index')
    Balance_Sheet.columns = ['Current_Year']
    Balance_Sheet = Balance_Sheet[5:-2]
    Balance_Sheet['As_%_revnues'] = Balance_Sheet/ income_statement['Current_Year'].iloc[0]

    #Forcasting for the next 5 years now
    Balance_Sheet['Next_Year_1'] =  income_statement['Next_Year_1']['revenue'] * Balance_Sheet['As_%_revnues']
    Balance_Sheet['Next_Year_2'] =  income_statement['Next_Year_2']['revenue'] * Balance_Sheet['As_%_revnues']
    Balance_Sheet['Next_Year_3'] =  income_statement['Next_Year_3']['revenue'] * Balance_Sheet['As_%_revnues'] 
    Balance_Sheet['Next_Year_4'] =  income_statement['Next_Year_4']['revenue'] * Balance_Sheet['As_%_revnues'] 
    Balance_Sheet['Next_Year_5'] =  income_statement['Next_Year_5']['revenue'] * Balance_Sheet['As_%_revnues']

    # Getting Cash Flows 
    FCF_Forecast = {}
    FCF_Forecast['Next_Year_1'] = {}
    FCF_Forecast['Next_Year_1']['netIncome'] = income_statement['Next_Year_1']['netIncome']
    FCF_Forecast['Next_Year_1']['Depreciation '] = income_statement['Next_Year_1']['depreciationAndAmortization'] - income_statement['Current_Year']['depreciationAndAmortization']
    FCF_Forecast['Next_Year_1']['Receivables '] = Balance_Sheet['Next_Year_1']['netReceivables'] - Balance_Sheet['Current_Year']['netReceivables']
    FCF_Forecast['Next_Year_1']['Inventory'] = Balance_Sheet['Next_Year_1']['inventory'] - Balance_Sheet['Current_Year']['inventory']
    FCF_Forecast['Next_Year_1']['Payables '] = Balance_Sheet['Next_Year_1']['accountPayables'] - Balance_Sheet['Current_Year']['accountPayables']
    FCF_Forecast['Next_Year_1']['CF_operations'] = FCF_Forecast['Next_Year_1']['netIncome'] + FCF_Forecast['Next_Year_1']['Depreciation '] + (FCF_Forecast['Next_Year_1']['Receivables '] * -1) + (FCF_Forecast['Next_Year_1']['Inventory'] *-1) + FCF_Forecast['Next_Year_1']['Payables ']
    FCF_Forecast['Next_Year_1']['CAPEX'] = Balance_Sheet['Next_Year_1']['propertyPlantEquipmentNet'] - Balance_Sheet['Current_Year']['propertyPlantEquipmentNet'] + income_statement['Next_Year_1']['depreciationAndAmortization']

    FCF_Forecast['Next_Year_1']['FCF'] = FCF_Forecast['Next_Year_1']['CAPEX'] + FCF_Forecast['Next_Year_1']['CF_operations']


    FCF_Forecast['Next_Year_2'] = {}
    FCF_Forecast['Next_Year_2']['netIncome'] = income_statement['Next_Year_2']['netIncome']

    FCF_Forecast['Next_Year_2']['Depreciation '] = income_statement['Next_Year_2']['depreciationAndAmortization'] - income_statement['Next_Year_1']['depreciationAndAmortization']
    FCF_Forecast['Next_Year_2']['Receivables '] = Balance_Sheet['Next_Year_2']['netReceivables'] - Balance_Sheet['Next_Year_1']['netReceivables']
    FCF_Forecast['Next_Year_2']['Inventory'] = Balance_Sheet['Next_Year_2']['inventory'] - Balance_Sheet['Next_Year_1']['inventory']
    FCF_Forecast['Next_Year_2']['Payables '] = Balance_Sheet['Next_Year_2']['accountPayables'] - Balance_Sheet['Next_Year_1']['accountPayables']
    FCF_Forecast['Next_Year_2']['CF_operations'] = FCF_Forecast['Next_Year_2']['netIncome'] + FCF_Forecast['Next_Year_2']['Depreciation '] + (FCF_Forecast['Next_Year_2']['Receivables '] * -1) + (FCF_Forecast['Next_Year_2']['Inventory'] *-1) + FCF_Forecast['Next_Year_2']['Payables ']
    FCF_Forecast['Next_Year_2']['CAPEX'] = Balance_Sheet['Next_Year_2']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_1']['propertyPlantEquipmentNet'] + income_statement['Next_Year_2']['depreciationAndAmortization']
    FCF_Forecast['Next_Year_2']['FCF'] = FCF_Forecast['Next_Year_2']['CAPEX'] + FCF_Forecast['Next_Year_2']['CF_operations']


    FCF_Forecast['Next_Year_3'] = {}
    FCF_Forecast['Next_Year_3']['netIncome'] = income_statement['Next_Year_3']['netIncome']

    FCF_Forecast['Next_Year_3']['Depreciation '] = income_statement['Next_Year_3']['depreciationAndAmortization'] - income_statement['Next_Year_2']['depreciationAndAmortization']
    FCF_Forecast['Next_Year_3']['Receivables '] = Balance_Sheet['Next_Year_3']['netReceivables'] - Balance_Sheet['Next_Year_2']['netReceivables']
    FCF_Forecast['Next_Year_3']['Inventory'] = Balance_Sheet['Next_Year_3']['inventory'] - Balance_Sheet['Next_Year_2']['inventory']
    FCF_Forecast['Next_Year_3']['Payables '] = Balance_Sheet['Next_Year_3']['accountPayables'] - Balance_Sheet['Next_Year_2']['accountPayables']
    FCF_Forecast['Next_Year_3']['CF_operations'] = FCF_Forecast['Next_Year_3']['netIncome'] + FCF_Forecast['Next_Year_3']['Depreciation '] + (FCF_Forecast['Next_Year_3']['Receivables '] * -1) + (FCF_Forecast['Next_Year_3']['Inventory'] *-1) + FCF_Forecast['Next_Year_3']['Payables ']
    FCF_Forecast['Next_Year_3']['CAPEX'] = Balance_Sheet['Next_Year_3']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_2']['propertyPlantEquipmentNet'] + income_statement['Next_Year_3']['depreciationAndAmortization']
    FCF_Forecast['Next_Year_3']['FCF'] = FCF_Forecast['Next_Year_3']['CAPEX'] + FCF_Forecast['Next_Year_3']['CF_operations']


    FCF_Forecast['Next_Year_4'] = {}
    FCF_Forecast['Next_Year_4']['netIncome'] = income_statement['Next_Year_4']['netIncome']

    FCF_Forecast['Next_Year_4']['Depreciation '] = income_statement['Next_Year_4']['depreciationAndAmortization'] - income_statement['Next_Year_3']['depreciationAndAmortization']
    FCF_Forecast['Next_Year_4']['Receivables '] = Balance_Sheet['Next_Year_4']['netReceivables'] - Balance_Sheet['Next_Year_3']['netReceivables']
    FCF_Forecast['Next_Year_4']['Inventory'] = Balance_Sheet['Next_Year_4']['inventory'] - Balance_Sheet['Next_Year_3']['inventory']
    FCF_Forecast['Next_Year_4']['Payables '] = Balance_Sheet['Next_Year_4']['accountPayables'] - Balance_Sheet['Next_Year_3']['accountPayables']
    FCF_Forecast['Next_Year_4']['CF_operations'] = FCF_Forecast['Next_Year_4']['netIncome'] + FCF_Forecast['Next_Year_4']['Depreciation '] + (FCF_Forecast['Next_Year_4']['Receivables '] * -1) + (FCF_Forecast['Next_Year_4']['Inventory'] *-1) + FCF_Forecast['Next_Year_4']['Payables ']
    FCF_Forecast['Next_Year_4']['CAPEX'] = Balance_Sheet['Next_Year_4']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_3']['propertyPlantEquipmentNet'] + income_statement['Next_Year_4']['depreciationAndAmortization']
    FCF_Forecast['Next_Year_4']['FCF'] = FCF_Forecast['Next_Year_4']['CAPEX'] + FCF_Forecast['Next_Year_4']['CF_operations']

    FCF_Forecast['Next_Year_5'] = {}
    FCF_Forecast['Next_Year_5']['netIncome'] = income_statement['Next_Year_5']['netIncome']

    FCF_Forecast['Next_Year_5']['Depreciation '] = income_statement['Next_Year_5']['depreciationAndAmortization'] - income_statement['Next_Year_4']['depreciationAndAmortization']
    FCF_Forecast['Next_Year_5']['Receivables '] = Balance_Sheet['Next_Year_5']['netReceivables'] - Balance_Sheet['Next_Year_4']['netReceivables']
    FCF_Forecast['Next_Year_5']['Inventory'] = Balance_Sheet['Next_Year_5']['inventory'] - Balance_Sheet['Next_Year_4']['inventory']
    FCF_Forecast['Next_Year_5']['Payables '] = Balance_Sheet['Next_Year_5']['accountPayables'] - Balance_Sheet['Next_Year_4']['accountPayables']
    FCF_Forecast['Next_Year_5']['CF_operations'] = FCF_Forecast['Next_Year_5']['netIncome'] + FCF_Forecast['Next_Year_5']['Depreciation '] + (FCF_Forecast['Next_Year_5']['Receivables '] * -1) + (FCF_Forecast['Next_Year_5']['Inventory'] *-1) + FCF_Forecast['Next_Year_5']['Payables ']
    FCF_Forecast['Next_Year_5']['CAPEX'] = Balance_Sheet['Next_Year_5']['propertyPlantEquipmentNet'] - Balance_Sheet['Next_Year_4']['propertyPlantEquipmentNet'] + income_statement['Next_Year_5']['depreciationAndAmortization']
    FCF_Forecast['Next_Year_5']['FCF'] = FCF_Forecast['Next_Year_5']['CAPEX'] + FCF_Forecast['Next_Year_5']['CF_operations']

    #converting the dictionary to Panda Dataset for better visualization
    FCF_Forecast_data = pd.DataFrame.from_dict(FCF_Forecast, orient ='columns')
    #Seperate the numbers to thousands for better formatting
    pd.options.display.float_format = '{:,.0f}'.format

    # NPV OF CASH FLOWS
    # The values below (WACC and PG will be simulated using Monte Carlo Simulation)
    #perpetual_growth = 0.02 #assuming 
    #WACC = 0.09 # assuming

    fcf_lst = FCF_Forecast_data.iloc[-1].values.tolist()
    npv = np.npv(WACC, fcf_lst)

    # Getting total number of a Company's Shares
    number_of_shares = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{company}?apikey={demo}').json()
    number_of_shares = number_of_shares[0]['numberOfShares']

    #Calculating the Terminal value 
    terminal_value = (FCF_Forecast['Next_Year_5']['FCF']*(1+perpetual_growth))/(WACC-perpetual_growth)
    terminal_value
    # Calculating the share price of the company 
    terminal_value_discounted = terminal_value/(1-WACC)**4
    company_value = terminal_value_discounted + npv
    share_value = company_value/number_of_shares
    #print("SHARE VALUE OF", company, '$',share_value)


In [112]:
Share_Price_FCF_Forc(company = 'GOOG', revenue_g = 0.02,perpetual_growth= 0.002,WACC= 0.07)
Share_Price_FCF_Forc(company = 'FB', revenue_g = 0.02,perpetual_growth= 0.002,WACC= 0.07)
Share_Price_FCF_Forc(company = 'NFLX', revenue_g = 0.02,perpetual_growth= 0.002,WACC= 0.07)

KeyError: 0

In [108]:
import numpy as np
import matplotlib.pyplot as plt
lst = []

for i in range(1):
    perp_growth = np.random.normal(0.002,0.001)
    growth = np.random.normal(0.02,0.01)
    wacc = np.random.normal(0.07,0.02)
    #lst.append(Share_Price_FCF_Forc(company = 'NFLX', revenue_g = growth,perpetual_growth= perp_growth,WACC= wacc))
    print(perp_growth)
    print(growth)
    print(wacc)



0.0009128791857674968
0.006661836001872697
0.07098083179353627


In [111]:
Share_Price_FCF_Forc(company = 'NFLX', revenue_g = 0.02,perpetual_growth= 0.002,WACC= 0.07)

KeyError: 0

In [18]:
import random
import matplotlib.pyplot as plt
import numpy as np

lsst = []

lsst.append(np.random.normal(1,10)) for i in range(10)
    
plt.hist(lsst)
plt.show()

SyntaxError: invalid syntax (<ipython-input-18-adac05a2bda7>, line 7)